In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
import seaborn as sns
import simplefix
from simplefix import FixMessage, FixParser
import socket
import ssl
import os

#### Credentials Setup

In [19]:
# Setup credentials for connection to cTrader
socket_buffer = 1024
server_url = "h35.p.ctrader.com"
server_port = 5211

sender = "demo.ctrader.3988979"
target = "CSERVER"
username = "3988979"
password = "SamaTrades123!"

#### Logs Setup

In [21]:
# Setup log file
logs_file = os.path.join('logs', 'trade', 'trade.log')
log_dir = os.path.join('logs', 'trade')

# Create directory for log file
if not os.path.isdir(log_dir):
    os.makedirs(log_dir)

def log(self,msg):
    log = open(logs_file, "a+")
    self.logs.write(datetime.now().strftime('%Y-%m-%d:%H:%M:%S')+' [Trade]'+msg)
    print('[Trade]'+msg)

#### SimpleFix Library Setup

In [97]:
# Create the FIX message parser
parser = simplefix.FixParser()
# Define a class to send messages to the FIX API
def send_ctrader_request(fix_message):
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.check_hostname = False
    context.verify_mode = ssl.CERT_NONE

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        with context.wrap_socket(sock, server_hostname=server_url) as ssock:
            # Create a socket connection and send the logon FIX message
            try:
                # Connect to the backend
                ssock.connect((server_url, server_port))
                # Send the message bytes to the backend
                ssock.send(api_message)
                # Fetch the API response
                response = ssock.recv(1024)
                # Use the parser to decode the API response
                parser.append_buffer(response)
                return parser.get_message()
            
            except Exception as e:
                print(f"An error occurred: {e}")
                
def generate_login_request():
    # Create the FIX message
    message = simplefix.FixMessage()
    message.append_pair(8, "FIX.4.4")
    message.append_pair(9, 126)
    message.append_pair(35, "A")
    message.append_pair(49, sender)
    message.append_pair(56, target)
    message.append_pair(34, 1, header=True)
    message.append_utc_timestamp(52)
    message.append_pair(57, "QUOTE")
    message.append_pair(98, 0)
    message.append_pair(108,30)
    message.append_pair(141, "Y")
    message.append_pair(553,username)
    message.append_pair(554,password)
    message.append_pair(10, 131)

    # Convert the FIX message to bytes
    return message.encode()

def generate_logout_request():
    #8=FIX.4.4|9=86|35=5|49=live.theBroker.12345|56=CSERVER|34=161|52=20170117-
    #09:22:33|57=TRADE|50=any_string|10=102|
    # Create the FIX message
    message = simplefix.FixMessage()
    message.append_pair(8, "FIX.4.4")
    message.append_pair(9, 126)
    message.append_pair(35, 5)
    message.append_pair(49, sender)
    message.append_pair(56, target)
    message.append_pair(34, 1, header=True)
    message.append_utc_timestamp(52)
    message.append_pair(57, "QUOTE")
    message.append_pair(98, 0)
    message.append_pair(108,30)
    message.append_pair(141, "Y")
    message.append_pair(553,username)
    message.append_pair(554,password)
    message.append_pair(10, 131)

    # Convert the FIX message to bytes
    return message.encode()

def generate_market_request():
    message = simplefix.FixMessage()
    message.append_pair(8, "FIX.4.4")
    message.append_pair(35, "V")
    message.append_pair(49, sender)
    message.append_pair(56, target)
    message.append_pair(34, 3, header=True)
    message.append_utc_timestamp(52)
    message.append_pair(50, "QUOTE")
    message.append_pair(262, 876316403)
    message.append_pair(263, 1)
    message.append_pair(264, 1)
    message.append_pair(265, 1)
    message.append_pair(267, 2)
    message.append_pair(55,"EURUSD")
    message.append_pair(10, 94)
    
    return message.encode()

def submit_order():
    message = simplefix.FixMessage()
    message.append_pair(8, "FIX.4.4")
    message.append_pair(9, "143")
    message.append_pair(35, "D")
    message.append_pair(49, sender)
    message.append_pair(56, target)
    message.append_pair(34, "77", header=True)
    message.append_utc_timestamp(52)
    message.append_pair(50, "any_string")
    message.append_pair(57, "TRADE")
    message.append_pair(11, "876316397")
    message.append_pair(55, "1")
    message.append_pair(54, "1")
    message.append_utc_timestamp(60, "20170117-10:02:14")
    message.append_pair(40, "1")
    message.append_pair(38, "10000")
    message.append_pair(10, "010")

    return message.encode()

def generate_order_status_request():
    message = simplefix.FixMessage()
    message.append_pair(8, "FIX.4.4")
    message.append_pair(9, "98")
    message.append_pair(35, "H")
    message.append_pair(49, sender)
    message.append_pair(56, target)
    message.append_pair(34, "95", header=True)
    message.append_utc_timestamp(52)
    message.append_pair(50, "any_string")
    message.append_pair(57, "TRADE")
    message.append_pair(11, "876316400")
    message.append_pair(10, "191")

    return message.encode()

def generate_request_for_positions():
    message = simplefix.FixMessage()
    message.append_pair(8, "FIX.4.4")
    message.append_pair(9, "100")
    message.append_pair(35, "AN")
    message.append_pair(49, sender)
    message.append_pair(56, target)
    message.append_pair(34, "99", header=True)
    message.append_utc_timestamp(52)
    message.append_pair(50, "any_string")
    message.append_pair(57, "TRADE")
    message.append_pair(710, "876316401")
    message.append_pair(10, "103")

    return message.encode()

In [99]:
login_response_message = send_ctrader_request(generate_login_request())

In [101]:
if(login_response_message.get(35).decode() == "A"):
    print("Login successful")
    
    market_data_response = send_ctrader_request(generate_market_request())
    print(market_data_response)
    
else:
    print(f"Login failed : {login_response_status.get(58)}")

Login successful
None
